## Setup

In [ ]:
import subprocess
import sys
from pathlib import Path

print(f"Python version: {sys.version}")
print(f"Working directory: {Path.cwd()}")

# Check if required directories exist
for dir_name in ['data', 'checkpoints', 'outputs', 'src']:
    dir_path = Path(dir_name)
    if dir_path.exists():
        print(f"✓ {dir_name}/ exists")
    else:
        print(f"✗ {dir_name}/ not found")
        dir_path.mkdir(parents=True, exist_ok=True)
        print(f"  Created {dir_name}/")

In [ ]:
# 의존성 설치 (필요시 실행)
INSTALL_DEPENDENCIES = False  # True로 변경하면 requirements.txt 설치

if INSTALL_DEPENDENCIES:
    print("Installing dependencies from requirements.txt...")
    result = subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], 
                          capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✓ Dependencies installed successfully")
    else:
        print("✗ Installation failed")
        print(result.stderr)
else:
    print("⊘ Dependency installation skipped")
    print("  (Set INSTALL_DEPENDENCIES = True to install)")
    
# Check key packages
print("\nChecking key packages...")
required_packages = ['torch', 'transformers', 'sklearn', 'tqdm', 'numpy', 'matplotlib']
missing_packages = []

for package in required_packages:
    try:
        __import__(package)
        print(f"  ✓ {package}")
    except ImportError:
        print(f"  ✗ {package} (missing)")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠ Missing packages: {', '.join(missing_packages)}")
    print("  Set INSTALL_DEPENDENCIES = True and re-run this cell")
else:
    print("\n✓ All required packages are available")

## 의존성 설치 (선택사항)

첫 실행 시 필요한 패키지를 설치합니다.

## 전체 실행 설정

In [ ]:
# 실행할 단계 선택 (True/False)
RUN_CONFIG = {
    'preprocess': True,           # 1. 데이터 전처리
    'train_baseline': True,       # 2. Baseline 모델 학습 (지도학습)
    'generate_text': True,        # 3. 대화 생성
    'train_multimodal': True,     # 4. Multimodal 모델 학습 (지도학습)
    'train_rl_baseline': True,    # 5. RL Baseline 모델 학습 (게임 상태만)
    'train_rl_multimodal': True,  # 6. RL Multimodal 모델 학습 (게임 상태 + 대화)
    'evaluate': True,             # 7. 전체 모델 평가
    
    # 평가 설정
    'n_eval_games': 1000,    # 평가할 게임 수
}

print("실행 설정:")
for key, value in RUN_CONFIG.items():
    print(f"  {key:20s}: {value}")

## Step 1: 데이터 전처리

Pluribus 데이터셋을 다운로드하고 377차원 feature로 변환합니다.

In [ ]:
if RUN_CONFIG['preprocess']:
    print("="*60)
    print("STEP 1: DATA PREPROCESSING")
    print("="*60)
    
    result = subprocess.run(['python', '1_preprocess_data.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 1 completed successfully")
    else:
        print("\n✗ Step 1 failed")
        print("Stopping pipeline...")
        raise RuntimeError("Preprocessing failed")
else:
    print("⊘ Step 1 skipped")

## Step 2: Baseline 모델 학습

게임 상태만 사용하는 MLP 모델을 학습합니다.

In [ ]:
if RUN_CONFIG['train_baseline']:
    print("="*60)
    print("STEP 2: TRAIN BASELINE MODEL")
    print("="*60)
    
    result = subprocess.run(['python', '2_train_baseline.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 2 completed successfully")
    else:
        print("\n✗ Step 2 failed")
        print("Stopping pipeline...")
        raise RuntimeError("Baseline training failed")
else:
    print("⊘ Step 2 skipped")

## Step 3: 대화 생성

LLM을 사용하여 각 게임 상태에 대한 대화를 생성합니다.

In [ ]:
if RUN_CONFIG['generate_text']:
    print("="*60)
    print("STEP 3: GENERATE DIALOGUES")
    print("="*60)
    
    result = subprocess.run(['python', '3_generate_dialogues.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 3 completed successfully")
    else:
        print("\n✗ Step 3 failed")
        print("Stopping pipeline...")
        raise RuntimeError("Dialogue generation failed")
else:
    print("⊘ Step 3 skipped")

## Step 4: Multimodal 모델 학습

게임 상태와 대화를 결합한 multimodal 모델을 학습합니다.

In [ ]:
if RUN_CONFIG['train_multimodal']:
    print("="*60)
    print("STEP 4: TRAIN MULTIMODAL MODEL")
    print("="*60)
    
    result = subprocess.run(['python', '4_train_multimodal.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 4 completed successfully")
    else:
        print("\n✗ Step 4 failed")
        print("Stopping pipeline...")
        raise RuntimeError("Multimodal training failed")
else:
    print("⊘ Step 4 skipped")

## Step 5: RL Baseline 모델 학습 (PPO)

PPO를 사용하여 게임 상태만으로 학습하는 강화학습 모델을 학습합니다.

In [ ]:
if RUN_CONFIG['train_rl_baseline']:
    print("="*60)
    print("STEP 5: TRAIN RL BASELINE MODEL (PPO)")
    print("="*60)
    
    result = subprocess.run(['python', '5_train_rl_baseline.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 5 completed successfully")
    else:
        print("\n✗ Step 5 failed")
        print("⚠ Continuing pipeline (RL training is optional)...")
else:
    print("⊘ Step 5 skipped")

## Step 6: RL Multimodal 모델 학습 (PPO)

PPO를 사용하여 게임 상태와 대화를 결합한 강화학습 모델을 학습합니다.

In [ ]:
if RUN_CONFIG['train_rl_multimodal']:
    print("="*60)
    print("STEP 6: TRAIN RL MULTIMODAL MODEL (PPO)")
    print("="*60)
    
    result = subprocess.run(['python', '6_train_rl_multimodal.py'], capture_output=False)
    
    if result.returncode == 0:
        print("\n✓ Step 6 completed successfully")
    else:
        print("\n✗ Step 6 failed")
        print("⚠ Continuing pipeline (RL training is optional)...")
else:
    print("⊘ Step 6 skipped")

## Step 7: 전체 모델 평가

학습된 모든 모델(지도학습 + 강화학습)을 rule-based agent와 대결시켜 평가합니다.

In [ ]:
if RUN_CONFIG['evaluate']:
    print("="*60)
    print("STEP 7: EVALUATE ALL MODELS")
    print("="*60)
    
    n_games = RUN_CONFIG['n_eval_games']
    results = {}
    
    # Evaluate Supervised Baseline
    print("\n--- Evaluating Supervised Baseline Model ---")
    result = subprocess.run([
        'python', '7_evaluate_vs_rule_based.py',
        '--model_type', 'baseline',
        '--model_path', 'checkpoints/baseline_best.pt',
        '--n_games', str(n_games),
        '--output_dir', 'outputs'
    ], capture_output=False)
    results['baseline'] = (result.returncode == 0)
    
    # Evaluate Supervised Multimodal
    print("\n--- Evaluating Supervised Multimodal Model ---")
    result = subprocess.run([
        'python', '7_evaluate_vs_rule_based.py',
        '--model_type', 'multimodal',
        '--model_path', 'checkpoints/multimodal_best.pt',
        '--n_games', str(n_games),
        '--output_dir', 'outputs'
    ], capture_output=False)
    results['multimodal'] = (result.returncode == 0)
    
    # Evaluate RL Baseline
    print("\n--- Evaluating RL Baseline Model ---")
    result = subprocess.run([
        'python', '7_evaluate_vs_rule_based.py',
        '--model_type', 'rl_baseline',
        '--model_path', 'checkpoints/rl_baseline_best.pt',
        '--n_games', str(n_games),
        '--output_dir', 'outputs'
    ], capture_output=False)
    results['rl_baseline'] = (result.returncode == 0)
    
    # Evaluate RL Multimodal
    print("\n--- Evaluating RL Multimodal Model ---")
    result = subprocess.run([
        'python', '7_evaluate_vs_rule_based.py',
        '--model_type', 'rl_multimodal',
        '--model_path', 'checkpoints/rl_multimodal_best.pt',
        '--n_games', str(n_games),
        '--output_dir', 'outputs'
    ], capture_output=False)
    results['rl_multimodal'] = (result.returncode == 0)
    
    # Summary
    success_count = sum(results.values())
    if success_count == len(results):
        print("\n✓ Step 7 completed successfully (all models evaluated)")
    elif success_count > 0:
        print(f"\n⚠ Step 7 completed with some errors ({success_count}/{len(results)} models evaluated)")
    else:
        print("\n✗ Step 7 failed (no models evaluated)")
else:
    print("⊘ Step 7 skipped")

## 결과 확인

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

print("="*60)
print("EVALUATION RESULTS SUMMARY")
print("="*60)

# Define model names and their file paths
models = {
    'Supervised Baseline': 'baseline_vs_rule_based.json',
    'Supervised Multimodal': 'multimodal_vs_rule_based.json',
    'RL Baseline': 'rl_baseline_vs_rule_based.json',
    'RL Multimodal': 'rl_multimodal_vs_rule_based.json'
}

output_dir = Path('outputs')
results = {}
available_models = []

# Load all available results
for model_name, filename in models.items():
    filepath = output_dir / filename
    if filepath.exists():
        with open(filepath, 'r') as f:
            results[model_name] = json.load(f)
            available_models.append(model_name)

if len(available_models) == 0:
    print("\n⚠ No evaluation results found")
    print("  Make sure to run evaluation steps (Step 5 and/or Step 8) first.")
else:
    # Display comparison table
    print(f"\n{'Model':<25s} {'Total Profit':>15s} {'Avg/Hand':>12s} {'Win Rate':>12s}")
    print("=" * 70)
    
    for model_name in available_models:
        result = results[model_name]
        total_profit = result['total_profit']
        avg_profit = result['avg_profit_per_hand']
        win_rate = result['win_rate']
        
        print(f"{model_name:<25s} {total_profit:>15.1f} {avg_profit:>12.2f} {win_rate:>11.1f}%")
    
    print("=" * 70)
    
    # Find best model by total profit
    best_model = max(available_models, key=lambda m: results[m]['total_profit'])
    best_profit = results[best_model]['total_profit']
    print(f"\n🏆 Best Model: {best_model} (Profit: {best_profit:+.1f} BB)")
    
    # Visualization
    if len(available_models) >= 2:
        fig, axes = plt.subplots(1, 3, figsize=(16, 5))
        
        # Plot 1: Total Profit
        ax = axes[0]
        profits = [results[m]['total_profit'] for m in available_models]
        colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12'][:len(available_models)]
        bars = ax.bar(range(len(available_models)), profits, color=colors, alpha=0.7, 
                     edgecolor='black', linewidth=2)
        ax.axhline(0, color='gray', linestyle='--', linewidth=1)
        ax.set_ylabel('Total Profit (BB)', fontsize=12, fontweight='bold')
        ax.set_title('Total Profit Comparison', fontsize=13, fontweight='bold')
        ax.set_xticks(range(len(available_models)))
        ax.set_xticklabels(available_models, rotation=15, ha='right')
        ax.grid(axis='y', alpha=0.3)
        
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}',
                    ha='center', va='bottom' if height > 0 else 'top',
                    fontweight='bold', fontsize=10)
        
        # Plot 2: Win Rate
        ax = axes[1]
        win_rates = [results[m]['win_rate'] for m in available_models]
        bars = ax.bar(range(len(available_models)), win_rates, color=colors, alpha=0.7,
                     edgecolor='black', linewidth=2)
        ax.set_ylabel('Win Rate (%)', fontsize=12, fontweight='bold')
        ax.set_title('Win Rate Comparison', fontsize=13, fontweight='bold')
        ax.set_xticks(range(len(available_models)))
        ax.set_xticklabels(available_models, rotation=15, ha='right')
        ax.set_ylim([0, 100])
        ax.grid(axis='y', alpha=0.3)
        
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.1f}%',
                    ha='center', va='bottom',
                    fontweight='bold', fontsize=10)
        
        # Plot 3: Avg Profit per Hand
        ax = axes[2]
        avg_profits = [results[m]['avg_profit_per_hand'] for m in available_models]
        bars = ax.bar(range(len(available_models)), avg_profits, color=colors, alpha=0.7,
                     edgecolor='black', linewidth=2)
        ax.axhline(0, color='gray', linestyle='--', linewidth=1)
        ax.set_ylabel('Avg Profit per Hand (BB)', fontsize=12, fontweight='bold')
        ax.set_title('Efficiency Comparison', fontsize=13, fontweight='bold')
        ax.set_xticks(range(len(available_models)))
        ax.set_xticklabels(available_models, rotation=15, ha='right')
        ax.grid(axis='y', alpha=0.3)
        
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.3f}',
                    ha='center', va='bottom' if height > 0 else 'top',
                    fontweight='bold', fontsize=10)
        
        plt.tight_layout()
        plt.savefig('outputs/all_models_comparison.png', dpi=150, bbox_inches='tight')
        print("\n📊 Visualization saved: outputs/all_models_comparison.png")
        plt.show()
    
    # Detailed comparison
    if len(available_models) >= 2:
        print("\n" + "="*70)
        print("DETAILED COMPARISON")
        print("="*70)
        
        # Compare supervised vs RL (if both available)
        if 'Supervised Baseline' in results and 'RL Baseline' in results:
            print("\n--- Baseline: Supervised vs RL ---")
            sup_profit = results['Supervised Baseline']['total_profit']
            rl_profit = results['RL Baseline']['total_profit']
            diff = rl_profit - sup_profit
            print(f"  Supervised: {sup_profit:+.1f} BB")
            print(f"  RL (PPO):   {rl_profit:+.1f} BB")
            print(f"  Difference: {diff:+.1f} BB ({diff/abs(sup_profit)*100 if sup_profit != 0 else 0:+.1f}%)")
        
        if 'Supervised Multimodal' in results and 'RL Multimodal' in results:
            print("\n--- Multimodal: Supervised vs RL ---")
            sup_profit = results['Supervised Multimodal']['total_profit']
            rl_profit = results['RL Multimodal']['total_profit']
            diff = rl_profit - sup_profit
            print(f"  Supervised: {sup_profit:+.1f} BB")
            print(f"  RL (PPO):   {rl_profit:+.1f} BB")
            print(f"  Difference: {diff:+.1f} BB ({diff/abs(sup_profit)*100 if sup_profit != 0 else 0:+.1f}%)")
        
        # Compare baseline vs multimodal effect
        if 'RL Baseline' in results and 'RL Multimodal' in results:
            print("\n--- RL Models: Baseline vs Multimodal ---")
            baseline_profit = results['RL Baseline']['total_profit']
            multimodal_profit = results['RL Multimodal']['total_profit']
            diff = multimodal_profit - baseline_profit
            print(f"  Baseline:   {baseline_profit:+.1f} BB")
            print(f"  Multimodal: {multimodal_profit:+.1f} BB")
            print(f"  Dialogue Impact: {diff:+.1f} BB ({diff/abs(baseline_profit)*100 if baseline_profit != 0 else 0:+.1f}%)")
        
        print("="*70)

## 완료!

전체 파이프라인이 실행되었습니다.

### 생성된 파일들
- `data/processed/` - 전처리된 데이터
- `data/text/` - 생성된 대화
- `checkpoints/` - 학습된 모델 (지도학습 + 강화학습)
- `outputs/` - 평가 결과 및 시각화

### 학습된 모델들
1. **Supervised Learning (지도학습)**
   - Baseline: 게임 상태만 사용 (MLP)
   - Multimodal: 게임 상태 + 대화 사용

2. **Reinforcement Learning (PPO)**
   - RL Baseline: 게임 상태만 사용 (Actor-Critic)
   - RL Multimodal: 게임 상태 + 대화 사용 (Actor-Critic)

### 다음 단계
- 4가지 모델 비교 분석
- 개별 스크립트를 직접 실행하여 하이퍼파라미터 조정
- RL 모델의 학습 곡선 분석 (outputs/rl_*_training.png)

## [optional] 훈련된 봇 상대 플레이

In [ ]:
subprocess.run(['python', '8_evaluate_vs_rule_based.py'])